<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Academics/AI%20Project/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as ureq
ureq.urlretrieve('https://raw.githubusercontent.com/TXH2020/fast-labeling-workflow/master/Datasets/webintents.json','intents.json')

('intents.json', <http.client.HTTPMessage at 0x7ac834f126b0>)

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
# things we need for Tensorflow
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training,dtype='object')

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
# reset underlying graph data
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.layers.Dense(8,input_shape=(len(train_x[0]),),kernel_initializer='normal'),
    keras.layers.Dense(8,kernel_initializer='normal'),
    keras.layers.Dense(len(train_y[0]),activation='softmax',kernel_initializer='normal')
])
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y), epochs=1000, batch_size=8, verbose=0)

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [12]:
p = bow("1st sem courses", words)
print (p)
print (classes)

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
['1st&2ndsem', '1styearsyll', '2ndyearsyll', '3rdsem', '3rdyearsyll', '4thsem', '4thyearsyll', '5thsem', '6thsem', '7thsem', '8thsem', 'ADD', 'AI', 'AP Maam', 'Akshata Maam', 'Bhushan Sir', 'BigData', 'CD', 'CNL', 'CNS', 'COA', 'Chandrika Maam', 'Chetan Sir', 'Cloud', 'DAA', 'DAALAB', 'DBMS', 'DCN', 'DL', 'DMML', 'DMS', 'DRB Sir', 'DS', 'Darshana Maam', 'Faculty Profiles', 'FinalProject', 'Ganesh Sir', 'Geeta Maam', 'HOD', 'HPC', 'IOT', 'Jamuna Maam', 'Java', 'Jayalakshmi Maam', 'Kamath Maam', 'MAD', 'MP', 'Mallegowda Sir', 'Mamatha Maam', 'Manjula Maam', 'MiniProject', 'M

In [13]:
print(model.predict(np.array([p])))

1/1 [==============================] - 0s 95ms/step
[[9.99527693e-01 1.57815867e-20 7.51073627e-18 4.65831889e-08
  1.95703793e-32 1.21517326e-06 0.00000000e+00 1.84546261e-05
  1.37313342e-04 5.86906843e-11 3.15237965e-04 8.10181931e-35
  1.32319527e-29 6.87635102e-33 2.44169109e-27 1.29578008e-17
  3.99016831e-19 0.00000000e+00 4.25608350e-23 5.67899406e-13
  1.20376863e-35 5.95503327e-24 5.14170920e-14 4.89284278e-29
  0.00000000e+00 0.00000000e+00 2.00846133e-38 4.99488344e-08
  1.00508853e-27 0.00000000e+00 4.57239965e-17 2.71242552e-19
  3.27920012e-23 6.02250054e-37 2.32617947e-23 0.00000000e+00
  3.93265558e-26 3.16065919e-22 1.09494895e-35 5.60384738e-24
  8.36359629e-19 1.84329988e-23 4.42442225e-33 1.34555649e-21
  1.88832747e-26 3.33088049e-29 2.52070646e-28 9.63157732e-20
  5.10698016e-30 7.84460639e-23 0.00000000e+00 5.29258812e-35
  2.79069381e-25 4.85116138e-27 7.52138958e-29 7.31770890e-35
  9.16289949e-34 2.92985814e-33 4.80388678e-13 3.82835530e-36
  0.00000000e+00 2

In [14]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('m.tflite', 'wb') as f:
  f.write(tflite_model)

In [15]:
import pickle
with open('i.pkl','wb') as f:
  pickle.dump(intents,f)
with open('c.pkl','wb') as f:
  pickle.dump(classes,f)
with open('w.pkl','wb') as f:
  pickle.dump(words,f)